# LSTM

Import des librairies

In [67]:
import numpy as np
import time
from datetime import datetime

Chargement des données

In [54]:
path = "/Users/gauthier/Desktop/PO/inputs/group2min_dtrain2h_dpred30min_window1h"

input_type = path[path.rfind('/')+1:]

X_train, y_train = np.load(path+"/X_train.npy", allow_pickle=True), np.load(path+"/y_train.npy", allow_pickle=True)

X_val, y_val = np.load(path+"/X_val.npy", allow_pickle=True), np.load(path+"/y_val.npy", allow_pickle=True)

X_test, y_test = np.load(path+"/X_test.npy", allow_pickle=True), np.load(path+"/y_test.npy", allow_pickle=True)

Données d'architecture du LSTM

In [55]:
nb_inputs = X_train.shape[0]
nb_steps = X_train.shape[1]
nb_features = X_train.shape[2]

nb_inputs

6679

## Keras

In [20]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import keras
import keras.backend as K
import tensorflow as tf

from livelossplot import PlotLossesKeras

# Load the TensorBoard notebook extension
%load_ext tensorboard

Using TensorFlow backend.


In [41]:
nb_steps = X_train.shape[1]
nb_features = X_train.shape[2]
nb_hidden_units = 400
batch_size = 256

out_shape = y_train.shape[1]

Metric to evaluate model

In [22]:
def count_by_row(t):
    y, idx, count = tf.unique_with_counts(t)
    return K.sum(count-1)

def metric_top_10(y_true, y_pred):
    _, ind_pred = tf.nn.top_k(y_pred, k=10, sorted=True, name=None)
    _, ind_true = tf.nn.top_k(y_true, k=10, sorted=True, name=None)

    count = tf.map_fn(count_by_row, tf.concat([ind_pred, ind_true], 1))
    return K.mean(count)
    # return tf.round(tf.reduce_mean(count), 3)

In [56]:
start=time.time()
model = Sequential()

model.add(LSTM(nb_hidden_units, activation='relu', input_shape=(nb_steps, nb_features)))
model.add(Dense(out_shape))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy', metric_top_10])

checkpointer = ModelCheckpoint(filepath='weights/input_{}_hidden_{}_batch_{}_{}'.format(input_type, nb_hidden_units, batch_size, datetime.now()), verbose=1, save_best_only=True)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs/input_{}_hidden_{}_batch_{}_{}'.format(input_type, nb_hidden_units, batch_size, datetime.now()),
                                                      histogram_freq=1, update_freq='epoch')
# Load pre trained weights
#model.load_weights('/tmp/weights.hdf5')

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1, 
          callbacks=[PlotLossesKeras(), checkpointer,  tensorboard_callback])

print("\n")
print("Time : {:.2f} s".format(time.time()-start))

Train on 6679 samples, validate on 743 samples
Epoch 1/100
4192/6679 [=================>............] - ETA: 24s - loss: 112923898272411.3438 - accuracy: 0.0332 - metric_top_10: 0.3130

KeyboardInterrupt: 

Tensorboard

In [36]:
# %tensorboard --logdir logs/

Validate on test set

In [45]:
y_pred = model.predict(X_test)

In [48]:
metric_top_10(y_test, y_pred).numpy()

5

In [58]:
time.date()

AttributeError: module 'time' has no attribute 'date'

In [69]:
print(datetime.now())

2020-01-22 10:17:58.499941
